In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel('eines.xlsx')
data.head(10)

,_time,SkidNr,Line,Camera,Recognized,Quality1,Quality2,Quality3,Quality4,Quality_Min
0,2022-06-27T14:30:04.000+0200,1322,3,6,OK,0.88,0.93,0.91,0.92,0.88
1,2022-06-27T14:29:58.000+0200,365,3,4,OK,0.97,0.93,0.97,0.89,0.89
2,2022-06-27T14:29:30.000+0200,329,2,4,OK,0.95,0.91,0.94,0.96,0.91
3,2022-06-27T14:29:27.000+0200,104,1,2,OK,0.96,0.84,0.96,0.93,0.84
4,2022-06-27T14:29:23.000+0200,576,2,6,OK,0.98,0.91,0.94,0.95,0.91
...,...,...,...,...,...,...,...,...,...,...
211258,2022-06-03T05:08:34.000+0200,990,2,2,OK,0.96,0.94,0.95,0.96,0.94
211259,2022-06-03T05:08:22.000+0200,366,2,4,OK,0.97,0.89,0.96,0.95,0.89
211260,2022-06-03T05:08:06.000+0200,442,2,6,OK,0.98,0.94,0.93,0.93,0.93
211261,2022-06-03T05:07:44.000+0200,981,3,2,OK,0.94,0.94,0.94,0.83,0.83


In [3]:
#parse column _time to datetime format
data['_time'] = pd.to_datetime(data['_time'])

data['Date'] = pd.to_datetime(data['_time']).dt.date
data['Time'] = pd.to_datetime(data['_time']).dt.time
second_column = data.pop('Date')
data.insert(1, 'Date', second_column)
third_column = data.pop('Time')
data.insert(2, 'Time', third_column)
data.pop('_time')

data


,Date,Time,SkidNr,Line,Camera,Recognized,Quality1,Quality2,Quality3,Quality4,Quality_Min
0,2022-06-27,14:30:04,1322,3,6,OK,0.88,0.93,0.91,0.92,0.88
1,2022-06-27,14:29:58,365,3,4,OK,0.97,0.93,0.97,0.89,0.89
2,2022-06-27,14:29:30,329,2,4,OK,0.95,0.91,0.94,0.96,0.91
3,2022-06-27,14:29:27,104,1,2,OK,0.96,0.84,0.96,0.93,0.84
4,2022-06-27,14:29:23,576,2,6,OK,0.98,0.91,0.94,0.95,0.91
...,...,...,...,...,...,...,...,...,...,...,...
211258,2022-06-03,05:08:34,990,2,2,OK,0.96,0.94,0.95,0.96,0.94
211259,2022-06-03,05:08:22,366,2,4,OK,0.97,0.89,0.96,0.95,0.89
211260,2022-06-03,05:08:06,442,2,6,OK,0.98,0.94,0.93,0.93,0.93
211261,2022-06-03,05:07:44,981,3,2,OK,0.94,0.94,0.94,0.83,0.83


In [4]:
#bin encoding of column data['Recognized']
#OK - 1, NOK - 0
data['Recognized'] = data['Recognized'].map(lambda x: 1 if x=='OK' else 0)
data.head(10)

,Date,Time,SkidNr,Line,Camera,Recognized,Quality1,Quality2,Quality3,Quality4,Quality_Min
0,2022-06-27,14:30:04,1322,3,6,1,0.88,0.93,0.91,0.92,0.88
1,2022-06-27,14:29:58,365,3,4,1,0.97,0.93,0.97,0.89,0.89
2,2022-06-27,14:29:30,329,2,4,1,0.95,0.91,0.94,0.96,0.91
3,2022-06-27,14:29:27,104,1,2,1,0.96,0.84,0.96,0.93,0.84
4,2022-06-27,14:29:23,576,2,6,1,0.98,0.91,0.94,0.95,0.91
5,2022-06-27,14:29:02,427,2,1,1,0.91,0.88,0.87,0.85,0.85
6,2022-06-27,14:28:55,506,1,3,1,0.95,0.90,0.96,0.96,0.90
7,2022-06-27,14:28:47,766,2,2,1,0.95,0.91,0.93,0.93,0.91
8,2022-06-27,14:28:43,1465,1,5,1,0.86,0.92,0.95,0.90,0.86
9,2022-06-27,14:28:33,189,3,2,1,0.94,0.83,0.93,0.93,0.83


In [5]:
# splitting data: 80% training and 20% test
features = list(data.columns.values.tolist())
X = data.loc[:, features]
y = data.loc[:, ['Recognized']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.2)

NameError: name 'train_test_split' is not defined